# Chapter 17: Design of Catalytic Reactors

## Example 17.1: Reactor_Development_Program.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-17, Example 1, Page 434
//Title: Reactor Development Program
//==========================================================================================================

clear
clc

//INPUT
dt=[0.081;0.205;3.6];//Reactor diameter for the three reactors in m
dte=[0.04;0.12;0.70];//Equivalent diameters for the three reactors in m
db=[0.05;0.057;0.07];//Estimated bubble size in the three reactors in m
Kr1=1.3889;//Kinetic constant for Reaction 1 in s^-1
Kr2=0.6111;//Kinetic constant for Reaction 2 in s^-1
Kr3=0.022;//Kinetic constant for Reaction 3 in s^-1
dp=60;//Particle size in micrometer
ephsilonm=0.50;//Void fraction of fixed bed
ephsilonmf=0.55;//Void fraction at minimum fluidized condition
umf=0.006;////Velocity at minimum fluidization condition in m/s
D=2E-5;//Diffusion coefficient of gas in m^2/s
gammab=0.005;//Ratio of volume of dispersed solids to that of bubble phase
uo=0.2;//Superficial gas velocity in m/s
XA=0.9;//Conversion
g=9.81;//Acceleration due to gravity in square m/s^2

//CALCULATION
Kr12=Kr1+Kr2;
n=length(dt);
i=1;
while i<=n
    //Preliminary Calcualtions
    ubr(i)=0.711*(g*db(i))^0.5;//Rise velocity of bubble from Eqn.(6.7)
    ub(i)=1.55*{(uo-umf)+14.1*(db(i)+0.005)}*dte(i)^0.32+ubr(i);//Bubble velocity for Geldart A particles from Equation from Eqn.(6.11)
    delta(i)=uo/ub(i);//Fraction of bed in bubbles from Eqn.(6.29)
    ephsilonf(i)=1-(1-delta(i))*(1-ephsilonmf);//Void fraction of fixed bed from Eqn.(6.20)
    fw=0.6;//Wake volume to bubble volume from Fig.(5.8)
    gammac(i)=(1-ephsilonmf)*((3/(ubr(i)*ephsilonmf/umf-1))+fw);//Volume of solids in cloud to that of the bubble from Eqn.(6.36)
    gammae(i)=((1-ephsilonmf)*((1-delta(i))/delta(i)))-gammab-gammac(i);//Volume of solids in emulsion to that of the bubble from Eqn.(6.35)
    Kbc(i)=4.5*(umf/db(i))+5.85*((D^0.5*g^0.25)/db(i)^(5/4));//Gas interchange coefficient between bubble and cloud from Eqn.(10.27)
    Kce(i)=6.77*((D*ephsilonmf*0.711*(g*db(i))^0.5)/db(i)^3)^0.5;//Gas interchange coefficient between emulsion and cloud from Eqn.(10.34)
    //Effective rate constant from Eqn.(12.32)
    Kf12(i)=(gammab*Kr12+1/((1/Kbc(i))+(1/(gammac(i)*Kr12+1/((1/Kce(i))+(1/(gammae(i)*Kr12)))))))*(delta(i)/(1-ephsilonf(i)));
    //Rate of reaction 2 for fluidized bed from Eqn.(12.14)
    Kf3(i)=(gammab*Kr3+1/((1/Kbc(i))+(1/(gammac(i)*Kr3+1/((1/Kce(i))+(1/(gammae(i)*Kr3)))))))*(delta(i)/(1-ephsilonf(i)));
    //Rate of raection with respect to A from Eqn.(12.35)
    KfA(i)=[[Kbc(i)*Kce(i)/gammac(i)^2+(Kr12+Kce(i)/gammac(i)+Kce(i)/gammae(i))*(Kr3+Kce(i)/gammac(i)+Kce(i)/gammae(i))]*delta(i)*Kbc(i)*Kr12*Kr3/(1-ephsilonf(i))]    /[[(Kr12+Kbc(i)/gammac(i))*(Kr12+Kce(i)/gammae(i))+Kr12*Kce(i)/gammac(i)]*[(Kr3+Kbc(i)/gammac(i))*(Kr3+Kce(i)/gammae(i))+Kr3*Kce(i)/gammac(i)]];
    KfAR(i)=((Kr1/Kr12)*Kf12(i))-KfA(i);//Rate of reaction from Eqn.(12.34)
    KfAR1(i)=((Kr1/Kr12)*Kf12(i));//Since KfA is small
    
    //(b)Relate Selectivity with conversion in three reactors
    x=-log(1-XA);//The term Kf12*tou in Eqn.(12.26)
    tou(i)=x/Kf12(i);//Residence time from Eqn.(12.26)
    y(i)=(KfAR1(i)/(Kf3(i)-Kf12(i)))*(exp(-x)-exp(-tou(i)*Kf3(i)));//CR/CAi from Eqn.(12.27)
    SR(i)=y(i)/XA;//Selectivity of R
    
    //(c)Relate exit composition to space time
    tou1=5;//Space time in s
    XA1(i)=1-exp(-Kf12(i)*tou1);//Conversion from Eqn.(12.26)
    y1(i)=((KfAR1(i)/(Kf12(i)-Kf3(i)))*[exp(-Kf3(i)*tou1)-exp(-Kf12(i)*tou1)]);//CR/CAi R from Eqn.(12.27)
    
    //(d)Calculate height of bed needed to maximize production
    y2(i)=(KfAR1(i)/Kf12(i))*(Kf12(i)/Kf3(i))^(Kf3(i)/(Kf3(i)-Kf12(i)));//CRmax/CAi R from Eqn.(12.37)
    tou2(i)=log(Kf3(i)/Kf12(i))/(Kf3(i)-Kf12(i));//Space time from Eqn.(38)
    Lf(i)=(uo/(1-ephsilonf(i)))*tou2(i);//Length of bed at fully fluidized condition from Eqn.(12.5)
    Lm(i)=Lf(i)*(1-ephsilonf(i))/(1-ephsilonm);//Length of bed when settled
    XA2(i)=1-exp(-Kf12(i)*tou2(i));//Conversion from Eqn.(12.26)
    i=i+1;
end

//OUTPUT
printf('
Let Laboratory, Pilot plant, Semicommercial unit be Reactor 1,2 & 3 respectively'); 
printf('
(a)Relation between effective rate constant(Kf12) to the gas flow rate(uo)');
printf('
	Reactor No.	Kf12(s^-1)	uo(m/s)');
i=1;
while i<=n
     mprintf('
	%1.0f',i);
     mprintf('		%f',Kf12(i));
     mprintf('	%f',uo);
     i=i+1;
end
printf('
(b)Relation between selectivity with conversion');
printf('
	Reactor No.	Kf12(s^-1)	SR(mol R formed/mol A reacted)');
i=1;
while i<=n
     mprintf('
	%1.0f',i);
     mprintf('		%f',Kf12(i));
     mprintf('	%f',SR(i));
     i=i+1;
end
printf('
(c)Relation between exit compostion and space time');
printf('
	Reactor No.	XA		CR/CAi');
i=1;
while i<=n
     mprintf('
	%1.0f',i);
     mprintf('		%f',XA1(i));
     mprintf('	%f',y1(i));
     i=i+1;
end
printf('
(d)Height of bed needed to maximize the production of acrylonitrile');
printf('
	Reactor No.	Lm(m)		XA');
i=1;
while i<=n
     mprintf('
	%1.0f',i);
     mprintf('		%f',Lm(i));
     mprintf('	%f',XA2(i));
     i=i+1;
end

//====================================END OF PROGRAM ======================================================

## Example 17.2: Design_of_a_Commercial_Acrylonitrile_Reactor.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-17, Example 2, Page 438
//Title: Design of a Commercial Acrylonitrile Reactor
//==========================================================================================================

clear
clc

//INPUT
deltaHr=5.15E8;//Heat of reaction in J/k mol
W=5E4;//Weight of acrylonitirle produced per 334-day year in tonnes
db=0.07;//Estimated bubble size in m
dte=0.7;//Equivalent diameter in m
Kf12=0.35;//Effective rate constant in s^-1 from Example 1
dp=60;//Particle size in micrometer
ephsilonm=0.50;//Void fraction of fixed bed
ephsilonmf=0.55;//Void fraction at minimum fluidized condition
T=460;//Temperature in reactor in degree C
Pr=2.5;//Pressure inside reactor in bar
//Feed gas composition
x1=1;//Propylene
x2=1.1;//Ammonia
x3=11;//Air
do1=0.08;//OD of heat exchanger tubes in m\nL=7;//Length of tubes in m
ho=300;//Outside heat transfer coefficient in W/m^2 K
hi=1800;//Inside heat transfer coefficient in W/m^2 K
Tc=253.4;//Temperature of coolant in degree C
pi=3.14;

//CALCULATION
//Preliminary calculation
uo=0.46;//Superficial gas velocity from Fig.E1(a) for the value of Kf12 & db
tou=8;//Space time from Fig.E2(b) for highest concentraion of product R
Lm=uo*tou/(1-ephsilonm);
y=0.58;//CR/CAi from Fig.E1(c) for the value of tou & Kf12
XA=0.95//From Fig.E1(c) for the value of tou & Kf12
SR=y/XA;//Selectivity of R

//Cross-sectional area of the reactor
P=W*10^3/(334*24*3600);//Production rate of acrylonitrile
F=(P/0.053)/(SR*XA/0.042);//Feed rate of propylene
V=((F*22.4*(T+273)*(x1+x2+x3))/(42*273*Pr));
At=V/uo;//Cross-sectional area of reactor needed for the fluidized bed

//Heat exchanger calculation
q=F*XA*deltaHr/42;//Rate of heat liberation in the reactor
U=(ho^-1+hi^-1)^-1;//Overall heat transfer coefficient
deltaT=T-Tc;//Driving force for heat transfer
Aw=q/(U*deltaT);//Heat exchanger area required to remove q
Nt=Aw/(pi*do1*L);
li1=(At/Nt)^0.5;//Pitch for square pitch arrangement
dte1=4*[li1^2-(pi/4)*do1^2]/(pi*do1);
if dte1>dte then li=(pi/4*dte*do1+pi/4*do1^2)^0.5;//Pitch if we add dummy tubes
end
f=li^2-pi/4*do1^2;//Fraction of bed cross section taken up by tubes
dt1=sqrt(4/pi*At/(1-f));//Reactor diameter including all its tubes

//OUTPUT
printf('
Superficial gas velocity=%fm/s',uo);
printf('
No. of %1.0fm tubes required=%1.0f',L,Nt);
printf('
Reactor diameter=%fm',dt1);

//====================================END OF PROGRAM ======================================================

## Example 17.3: Reactor_Regenerator_with_Circulating_Catalyst_Catalytic_Cracking.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-17, Example 3, Page 444
//Title: Reactor-Regenerator with Circulating Catalyst: Catalytic Cracking
//==========================================================================================================

clear
clc

//INPUT
db=0.08;//Estimated bubble size in m
dte=2;//Equivalent diameter in m
F1=55.6;//Feed rate of oil in kg/s
XA=0.63;//Conversion
uo=0.6;//Superficial gas velocity in m/s
T1=500;//Temperature of reactor in degree C
T2=580;//Temperature of regenerator in degree C
Fs=F1*23.3;//Solid circulation rate from Ex.(15.2)
rhos=1200;//Density of catalyst in kg/m^3
dpbar=60;//Average particle size in micrometer
ephsilonm=0.50;//Void fraction of fixed bed
ephsilonmf=0.55;//Void fraction at minimum fluidized condition
umf=0.006;////Velocity at minimum fluidization condition in m/s
dt=8;//Diameter of reactor in m
D=2E-5;//Diffusion coefficient of gas in m^2/s
Kr=8.6;//Rate constant for reaction at 500 degree C in s^-1
Ka1=0.06;//Rate constant for deactivatiion at 500 degree C in s^-1
Ka2=0.012;//Rate constant for regeneration at 580 degree C in s^-1
gammab=0.005;//Ratio of volume of dispersed solids to that of bubble phase
g=9.81;//Acceleration due to gravity in square m/s^2
pi=3.14;

//CALCULATION
//Parameters for the fluidized reactor
ubr=0.711*(g*db)^0.5;//Rise velocity of bubble from Eqn.(6.7)
ub=1.55*{(uo-umf)+14.1*(db+0.005)}*dte^0.32+ubr;//Bubble velocity for Geldart A particles from Equation from Eqn.(6.11)
delta=uo/ub;//Fraction of bed in bubbles from Eqn.(6.29)
ephsilonf=1-(1-delta)*(1-ephsilonmf);//Void fraction of fixed bed from Eqn.(6.20)
fw=0.6;//Wake volume to bubble volume from Fig.(5.8)
gammac=(1-ephsilonmf)*((3/(ubr*ephsilonmf/umf-1))+fw);//Volume of solids in cloud to that of the bubble from Eqn.(6.36)
gammae=((1-ephsilonmf)*((1-delta)/delta))-gammab-gammac;//Volume of solids in emulsion to that of the bubble from Eqn.(6.35)
Kbc=4.5*(umf/db)+5.85*((D^0.5*g^0.25)/db^(5/4));//Gas interchange coefficient between bubble and cloud from Eqn.(10.27)
Kce=6.77*((D*ephsilonmf*0.711*(g*db)^0.5)/db^3)^0.5;//Gas interchange coefficient between emulsion and cloud from Eqn.(10.34)

//Bed height versus catalyst activity in reactor
a1bar=0.07;//Guess value for average activity in reactor
x=Kr*a1bar;//Value of Kra1 to be used in the following equation
Kf=(gammab*x+1/((1/Kbc)+(1/(gammac*x+1/((1/Kce)+(1/(gammae*x)))))))*(delta/(1-ephsilonf));//Effective rate constant from Eqn.(12.14)
tou=-log(1-XA)/Kf;//Space time from Eqn.(12.16)
Lm=tou*uo/(1-ephsilonm);//Length of fixed bed for guess value of a1bar
a1bar1=[0.0233;0.0465;0.0698;0.0930;0.116;0.140];//Various activity values to find Lm
n=length(a1bar1);
i=1;
while i<=n
    x1(i)=Kr*a1bar1(i);
    Kf1(i)=(gammab*x1(i)+1/((1/Kbc)+(1/(gammac*x1(i)+1/((1/Kce)+(1/(gammae*x1(i))))))))*(delta/(1-ephsilonf));//Effective rate constant from Eqn.(12.14)
    tou1(i)=-log(1-XA)/Kf1(i);//Space time from Eqn.(12.16)
    Lm1(i)=tou1(i)*uo/(1-ephsilonm);//Length of fixed bed for guess value of a1bar...Condition (i)
    i=i+1;
end

//Find the optimum size ratio for various a1bar
Lm=[5;6;7;8;10;12];
m=length(Lm);
i=1;
while i<=m
    W1(i)=(pi/4)*dt^2*rhos*(1-ephsilonm)*Lm(i);//Bed weight
    t1bar(i)=W1(i)/Fs;//Mean residence time of solids in reactor
    t2bar(i)=t1bar(i)*(Ka1/Ka2)^0.5;//Mean residence time of soilds at optimum from Eqn.(16)
    a1bar2(i)=(Ka2*t2bar(i))/(Ka1*t1bar(i)+Ka1*t1bar(i)*Ka2*t2bar(i)+Ka2*t2bar(i));//From Eqn.(15)...Condition (ii)
    i=i+1;
end

//Final design values
Lm4=7.3;//For satisfying condition (i) & (ii)
a1bar3=0.0744;//By interpolation
x2=a1bar3*Kr;
W11=(pi/4)*dt^2*rhos*(1-ephsilonm)*Lm4;//Bed weight for reactor
t1bar1=W11/Fs;//Mean residence time of solids in reactor
a2bar=(1+Ka1*t1bar1)*a1bar3;//Average activity in regenrator from Eqn.(10)
t2bar1=t1bar1*(Ka1/Ka2)^0.5;//Mean residence time of solids in regenerator from Eqn.(16)
W2=W11*(t2bar1/t1bar1);//Bed weight for regenerator
dt2=dt*(W2/W11)^0.5;//Diameter of regenerator assuming same static bed height for reactor and regerator

//OUTPUT
printf('
Bed height versus catalyst activity in reactor');
printf('
	Average activity');
printf('	Length of fixed bed(m)');
i=1;
while i<=n
    mprintf('
	%f',a1bar1(i));
    mprintf('		%f',Lm1(i));
    i=i+1;
end
printf('
Optimum size ratio for various activity in reactor');
printf('
	Length of fixed bed(m)');
printf('	Average activity');
i=1;
while i<=m
    mprintf('
	%f',Lm(i));
    mprintf('		%f',a1bar2(i));
    i=i+1;
end
printf('
Final design values');
printf('
	Diameter of reactor(m):%f',dt);
printf('
	Bed weight for reactor(tons):%f',W11/10^3);
printf('
	Bed weight for regenerator(tons):%f',W2/10^3);
printf('
	Diameter of regenerator(m):%f',dt2);
printf('
	Solid circulation rate(tons/hr):%f',Fs*3.6);

//====================================END OF PROGRAM ======================================================